In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np.random.seed(42)

In [0]:
data = pd.read_csv("train.csv")

In [0]:
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [0]:
data['Sex'] = pd.get_dummies(data['Sex'])
data['Embarked'] = pd.get_dummies(data['Embarked'])

In [85]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


In [86]:
data.isnull().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Embarked    False
dtype: bool

In [0]:
data.fillna(value=data['Age'].mean(), inplace=True)

In [0]:
dataX = data.drop(['Survived'], axis=1).values
dataY = data['Survived'].values

In [0]:
scaler = StandardScaler()
dataX = scaler.fit_transform(dataX)


In [0]:
trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=.2)

In [0]:
X = tf.placeholder(tf.float32, shape=(None, 7))
y = tf.placeholder(tf.int64, shape=(None))

In [0]:
with tf.name_scope("nn"):
  input_layer = tf.layers.dense(X, 7, tf.nn.relu)
  hidden1 = tf.layers.dense(input_layer, 300, tf.nn.relu)
  hidden2 = tf.layers.dense(hidden1, 100, tf.nn.relu)
  output = tf.layers.dense(hidden2, 2, tf.nn.relu)

In [0]:
with tf.name_scope("loss"):
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output)
  loss = tf.reduce_mean(cross_entropy)

In [0]:
with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval"):
  correct = tf.nn.in_top_k(output, y, 1)
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [97]:
with tf.Session() as sess:
  init.run()
  
  for i in range(1, 10001):
    sess.run(train_op, feed_dict={X:trainX, y:trainY.reshape(-1, 1)})
    if i % 1000 == 0:
      train_acc = accuracy.eval(feed_dict={X:trainX, y:trainY})
      test_acc = accuracy.eval(feed_dict={X:testX, y:testY})
      print(i // 1000)
      
  saver.save(sess, "/tmp/model.ckpt")

1
2
3
4
5
6
7
8
9
10


In [98]:
train_acc, test_acc

(1.0, 1.0)

In [0]:
test = pd.read_csv("test.csv")

In [119]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [0]:
test['Sex'] = pd.get_dummies(test['Sex'])
test['Embarked'] = pd.get_dummies(test['Embarked'])

In [0]:
test['Age'].fillna(value=test['Age'].mean(), inplace=True)
test['Fare'].fillna(value=test['Fare'].mean(), inplace=True)

In [0]:
x = test.values

In [0]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [112]:
with tf.Session() as sess:
  saver.restore(sess, "/tmp/model.ckpt")
  z = output.eval(feed_dict={X:x})
  preds = np.argmax(z, axis=1)

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


In [124]:
with open("/tmp/preds.csv", "w+") as f:
  counter = 0
  f.write("PassengerId,Survived\n")
  for i in preds:
    print(892 + counter)
    f.write("{},{}\n".format((892 + counter), i))
    counter += 1

892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
111